In [2]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [3]:
df = pd.read_csv('matches.csv', index_col=0)
df.shape

(1302, 27)

In [4]:
df.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team'],
      dtype='object')

In [5]:
df['team'].value_counts()

Arsenal                     66
Wolverhampton Wanderers     66
Southampton                 66
Everton                     66
Crystal Palace              66
Tottenham Hotspur           66
Leicester City              65
Brentford                   65
Manchester United           65
Chelsea                     65
Aston Villa                 65
Manchester City             65
Leeds United                65
West Ham United             64
Liverpool                   64
Brighton and Hove Albion    64
Newcastle United            64
Burnley                     38
Watford                     38
Norwich City                38
Nottingham Forest           27
Bournemouth                 27
Fulham                      27
Name: team, dtype: int64

Here all the teams do not have the same number of matches since it combines 2 seasons and some of them only played one of them.

Burnley, Watford and Norwich played in the previous season but got relegated and hence got 38 matches whereas Forest, Bournemouth and Fulham are playing only the current season and hence got 27 matches.

In [6]:
df['date'] = pd.to_datetime(df['date'])
df['venue_code'] = df['venue'].astype('category').cat.codes
df['opponent_code'] = df['opponent'].astype('category').cat.codes
df['day_code'] = df['date'].dt.dayofweek
df['month_code'] = df['date'].dt.month
df['hour_code'] = df['time'].str.split(':').str[0].astype(int)
df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opponent_code,day_code,month_code,hour_code
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,1.0,0.0,0.0,2022,Arsenal,0,7,4,8,20
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,0.0,0.0,0.0,2022,Arsenal,1,11,5,8,15
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,0.0,0.0,0.0,2022,Arsenal,0,2,5,8,17
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,1.0,0.0,0.0,2022,Arsenal,1,9,5,8,17
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,1.0,0.0,0.0,2022,Arsenal,1,1,2,8,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Away,L,0.0,2.0,Aston Villa,...,0.0,0.0,0.0,2021,Norwich City,0,1,5,4,15
39,2022-05-08,14:00,Premier League,Matchweek 36,Sun,Home,L,0.0,4.0,West Ham,...,1.0,0.0,0.0,2021,Norwich City,1,21,6,5,14
40,2022-05-11,19:45,Premier League,Matchweek 21,Wed,Away,L,0.0,3.0,Leicester City,...,0.0,0.0,0.0,2021,Norwich City,0,11,2,5,19
41,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Away,D,1.0,1.0,Wolves,...,0.0,0.0,0.0,2021,Norwich City,0,22,6,5,14


In [7]:
df["target"] = (df["result"] == "W").astype("int")

So now we have created input and target variables for our random forest model.

In [8]:
model = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=1)
train_x = df[df['date'] < '2022-11-11']
test_x = df[df['date'] >= '2022-11-11']
predictors = ['venue_code', 'opponent_code', 'day_code', 'month_code', 'hour_code']
model.fit(train_x[predictors], train_x['target'])

RandomForestClassifier(min_samples_split=10, random_state=1)

In [9]:
pred = model.predict(test_x[predictors])
acc = accuracy_score(test_x['target'], pred)
acc

0.5703703703703704

In [10]:
combined = pd.DataFrame({'actual': test_x['target'], 'predicted': pred})
pd.crosstab(combined['actual'], combined['predicted'])

predicted,0,1
actual,,
0,121,46
1,70,33


In [11]:
from sklearn.metrics import precision_score
wins = precision_score(test_x['target'], pred)
wins

0.4177215189873418

As we can see the accuracy of this model is 0.57 which is not bad but we can do better. And the team only wins 41% of the time which is not good. So lets improve our precision by rolling averages.

In [12]:
grouped = df.groupby('team')
group = grouped.get_group('Arsenal')
group 

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,pk,pkatt,season,team,venue_code,opponent_code,day_code,month_code,hour_code,target
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,0.0,0.0,2022,Arsenal,0,7,4,8,20,1
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,0.0,0.0,2022,Arsenal,1,11,5,8,15,1
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,0.0,0.0,2022,Arsenal,0,2,5,8,17,1
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,0.0,0.0,2022,Arsenal,1,9,5,8,17,1
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,0.0,0.0,2022,Arsenal,1,1,2,8,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,2022-05-01,16:30,Premier League,Matchweek 35,Sun,Away,W,2.0,1.0,West Ham,...,0.0,0.0,2021,Arsenal,0,21,6,5,16,1
41,2022-05-08,14:00,Premier League,Matchweek 36,Sun,Home,W,2.0,1.0,Leeds United,...,0.0,0.0,2021,Arsenal,1,10,6,5,14,1
42,2022-05-12,19:45,Premier League,Matchweek 22,Thu,Away,L,0.0,3.0,Tottenham,...,0.0,0.0,2021,Arsenal,0,19,3,5,19,0
43,2022-05-16,20:00,Premier League,Matchweek 37,Mon,Away,L,0.0,2.0,Newcastle Utd,...,0.0,0.0,2021,Arsenal,0,15,0,5,20,0


In [13]:
def rolling_averages(group, cols, new_col):
    group = group.sort_values('date')
    rollings = group[cols].rolling(3, closed='left').mean()
    group[new_col] = rollings
    #group = group.dropna(subset=[new_col])
    return group

In [14]:
group.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team', 'venue_code', 'opponent_code', 'day_code',
       'month_code', 'hour_code', 'target'],
      dtype='object')

In [15]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{col}_avg" for col in cols]
avgs = rolling_averages(group, cols, new_cols)
avgs

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,hour_code,target,gf_avg,ga_avg,sh_avg,sot_avg,dist_avg,fk_avg,pk_avg,pkatt_avg
0,2021-08-13,20:00,Premier League,Matchweek 1,Fri,Away,L,0.0,2.0,Brentford,...,20,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-22,16:30,Premier League,Matchweek 2,Sun,Home,L,0.0,2.0,Chelsea,...,16,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Away,L,0.0,5.0,Manchester City,...,12,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,1.0,0.0,Norwich City,...,15,1,0.000000,3.000000,9.666667,2.333333,14.833333,0.333333,0.000000,0.000000
5,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Away,W,1.0,0.0,Burnley,...,15,1,0.333333,2.333333,12.333333,3.000000,14.133333,0.333333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,2023-02-25,15:00,Premier League,Matchweek 25,Sat,Away,W,1.0,0.0,Leicester City,...,15,1,2.000000,2.000000,17.333333,4.000000,18.166667,0.000000,0.333333,0.333333
33,2023-03-01,19:45,Premier League,Matchweek 7,Wed,Home,W,4.0,0.0,Everton,...,19,1,2.000000,1.666667,13.000000,2.333333,18.100000,0.333333,0.333333,0.333333
34,2023-03-04,15:00,Premier League,Matchweek 26,Sat,Home,W,3.0,2.0,Bournemouth,...,15,1,3.000000,0.666667,15.000000,4.000000,16.000000,0.333333,0.000000,0.000000
36,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Away,W,3.0,0.0,Fulham,...,14,1,2.666667,0.666667,18.666667,5.333333,16.800000,1.000000,0.000000,0.000000


In [16]:
avgs = avgs.dropna(subset=new_cols)
avgs

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,hour_code,target,gf_avg,ga_avg,sh_avg,sot_avg,dist_avg,fk_avg,pk_avg,pkatt_avg
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,1.0,0.0,Norwich City,...,15,1,0.000000,3.000000,9.666667,2.333333,14.833333,0.333333,0.000000,0.000000
5,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Away,W,1.0,0.0,Burnley,...,15,1,0.333333,2.333333,12.333333,3.000000,14.133333,0.333333,0.000000,0.000000
7,2021-09-26,16:30,Premier League,Matchweek 6,Sun,Home,W,3.0,1.0,Tottenham,...,16,1,0.666667,1.666667,14.666667,3.000000,14.800000,0.666667,0.000000,0.000000
8,2021-10-02,17:30,Premier League,Matchweek 7,Sat,Away,D,0.0,0.0,Brighton,...,17,0,1.666667,0.333333,18.333333,5.333333,18.433333,0.666667,0.000000,0.000000
9,2021-10-18,20:00,Premier League,Matchweek 8,Mon,Home,D,2.0,2.0,Crystal Palace,...,20,0,1.333333,0.333333,11.000000,4.000000,19.833333,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,2023-02-25,15:00,Premier League,Matchweek 25,Sat,Away,W,1.0,0.0,Leicester City,...,15,1,2.000000,2.000000,17.333333,4.000000,18.166667,0.000000,0.333333,0.333333
33,2023-03-01,19:45,Premier League,Matchweek 7,Wed,Home,W,4.0,0.0,Everton,...,19,1,2.000000,1.666667,13.000000,2.333333,18.100000,0.333333,0.333333,0.333333
34,2023-03-04,15:00,Premier League,Matchweek 26,Sat,Home,W,3.0,2.0,Bournemouth,...,15,1,3.000000,0.666667,15.000000,4.000000,16.000000,0.333333,0.000000,0.000000
36,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Away,W,3.0,0.0,Fulham,...,14,1,2.666667,0.666667,18.666667,5.333333,16.800000,1.000000,0.000000,0.000000


In [17]:
#rolling averages for each team
team_avgs = df.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))
team_avgs = team_avgs.dropna(subset=new_cols)
team_avgs

date   time            comp         round  \
team                                                                         
Arsenal                 4  2021-09-11  15:00  Premier League   Matchweek 4   
                        5  2021-09-18  15:00  Premier League   Matchweek 5   
                        7  2021-09-26  16:30  Premier League   Matchweek 6   
                        8  2021-10-02  17:30  Premier League   Matchweek 7   
                        9  2021-10-18  20:00  Premier League   Matchweek 8   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 29 2023-02-24  20:00  Premier League  Matchweek 25   
                        30 2023-03-01  20:00  Premier League   Matchweek 7   
                        31 2023-03-04  15:00  Premier League  Matchweek 26   
                        32 2023-03-12  16:30  Premier League  Matchweek 27   
                        33 2023-03-18  15:00  Premier League  Matchweek 28   

                            day venue result   gf   ga        opponent  ...  \
team                                                                    ...   
Arsenal                 4   Sat  Home      W  1.0  0.0    Norwich City  ...   
                        5   Sat  Away      W  1.0  0.0         Burnley  ...   
                        7   Sun  Home      W  3.0  1.0       Tottenham  ...   
                        8   Sat  Away      D  0.0  0.0        Brighton  ...   
                        9   Mon  Home      D  2.0  2.0  Crystal Palace  ...   
...                         ...   ...    ...  ...  ...             ...  ...   
Wolverhampton Wanderers 29  Fri  Away      D  1.0  1.0          Fulham  ...   
                        30  Wed  Away      L  0.0  2.0       Liverpool  ...   
                        31  Sat  Home      W  1.0  0.0       Tottenham  ...   
                        32  Sun  Away      L  1.0  2.0   Newcastle Utd  ...   
                        33  Sat  Home      L  2.0  4.0    Leeds United  ...   

                            hour_code  target    gf_avg    ga_avg     sh_avg  \
team                                                                           
Arsenal                 4          15       1  0.000000  3.000000   9.666667   
                        5          15       1  0.333333  2.333333  12.333333   
                        7          16       1  0.666667  1.666667  14.666667   
                        8          17       0  1.666667  0.333333  18.333333   
                        9          20       0  1.333333  0.333333  11.000000   
...                               ...     ...       ...       ...        ...   
Wolverhampton Wanderers 29         20       0  1.666667  0.666667  12.666667   
                        30         20       0  1.000000  1.000000  11.333333   
                        31         15       1  0.333333  1.333333   9.000000   
                        32         16       0  0.666667  1.000000   6.666667   
                        33         15       0  0.666667  1.333333   6.333333   

                             sot_avg   dist_avg    fk_avg pk_avg  pkatt_avg  
team                                                                         
Arsenal                 4   2.333333  14.833333  0.333333    0.0        0.0  
                        5   3.000000  14.133333  0.333333    0.0        0.0  
                        7   3.000000  14.800000  0.666667    0.0        0.0  
                        8   5.333333  18.433333  0.666667    0.0        0.0  
                        9   4.000000  19.833333  0.666667    0.0        0.0  
...                              ...        ...       ...    ...        ...  
Wolverhampton Wanderers 29  3.333333  16.766667  0.000000    0.0        0.0  
                        30  2.333333  16.700000  0.000000    0.0        0.0  
                        31  2.333333  14.133333  0.000000    0.0        0.0  
                        32  2.666667  16.700000  0.000000    0.0        0.0  
             

In [18]:
team_avgs.index = range(team_avgs.shape[0])
team_avgs.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'season', 'team', 'venue_code', 'opponent_code', 'day_code',
       'month_code', 'hour_code', 'target', 'gf_avg', 'ga_avg', 'sh_avg',
       'sot_avg', 'dist_avg', 'fk_avg', 'pk_avg', 'pkatt_avg'],
      dtype='object')

In [19]:
def make_predictions(data, predictors):
    train = data[data['date'] < '2022--11-11']
    test = data[data['date'] >= '2022-11-11']
    model.fit(train[predictors], train['target'])
    pred = model.predict(test[predictors])
    combined = pd.DataFrame({'actual': test['target'], 'predicted': pred}, index=test.index)
    wins = precision_score(test['target'], pred)
    return combined, wins

In [20]:
combined, wins = make_predictions(team_avgs, predictors + new_cols)
wins

0.5555555555555556

In [21]:
combined

,actual,predicted
48,1,1
49,1,1
50,1,1
51,0,1
52,1,0
...,...,...
1228,0,0
1229,0,0
1230,1,1
1231,0,0


So our precision increased from 41% to 55% which is a decent improvement. 

In [22]:
combined = combined.merge(team_avgs[['team', 'opponent', 'date', 'result']], left_index=True, right_index=True)
combined

,actual,predicted,team,opponent,date,result
48,1,1,Arsenal,Wolves,2022-11-12,W
49,1,1,Arsenal,West Ham,2022-12-26,W
50,1,1,Arsenal,Brighton,2022-12-31,W
51,0,1,Arsenal,Newcastle Utd,2023-01-03,D
52,1,0,Arsenal,Tottenham,2023-01-15,W
...,...,...,...,...,...,...
1228,0,0,Wolverhampton Wanderers,Fulham,2023-02-24,D
1229,0,0,Wolverhampton Wanderers,Liverpool,2023-03-01,L
1230,1,1,Wolverhampton Wanderers,Tottenham,2023-03-04,W
1231,0,0,Wolverhampton Wanderers,Newcastle Utd,2023-03-12,L


In [25]:
map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", 
              "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", 
              "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"}

combined = combined.replace({"team": map_values, "opponent": map_values})
combined

,actual,predicted,team,opponent,date,result
48,1,1,Arsenal,Wolves,2022-11-12,W
49,1,1,Arsenal,West Ham,2022-12-26,W
50,1,1,Arsenal,Brighton,2022-12-31,W
51,0,1,Arsenal,Newcastle Utd,2023-01-03,D
52,1,0,Arsenal,Tottenham,2023-01-15,W
...,...,...,...,...,...,...
1228,0,0,Wolves,Fulham,2023-02-24,D
1229,0,0,Wolves,Liverpool,2023-03-01,L
1230,1,1,Wolves,Tottenham,2023-03-04,W
1231,0,0,Wolves,Newcastle Utd,2023-03-12,L


In [ ]:
combined.to_csv('predictions.csv')